In [1]:

# Se importa la libreria
import sqlite3

# Se conecta a la base de datos
conn = sqlite3.connect('bank.db')
cursor = conn.cursor()

# Creamos la tabla de usuarios si no existen
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        username TEXT PRIMARY KEY,
        password TEXT,
        balance REAL DEFAULT 2000.0,
        login_attempts INTEGER DEFAULT 0
    )
''')
conn.commit()

def login():
    # Se solicita usuario y contraseña
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    # Se comprueba si el usuario existe en la base de datos
    cursor.execute('SELECT * FROM users WHERE username = ?', (username,))
    user = cursor.fetchone()

    if user is None:
        print("Invalid username")
        return False

    if user[1] != password:
        
        # Incrementamos los intentos de inicio de sesion
        cursor.execute('UPDATE users SET login_attempts = login_attempts + 1 WHERE username = ?', (username,))
        conn.commit()

        if user[3] >= 2:
            print("Account locked. Please contact customer support.")
            return False
        else:
            print("Invalid password")
            return False

    # restablecemos los inicios de sesion
    cursor.execute('UPDATE users SET login_attempts = 0 WHERE username = ?', (username,))
    conn.commit()

    return True

def deposit():
    
    # Preguntar a usuario cuanto dinero va a depositar
    amount = float(input("Enter the amount to deposit: "))

    # Actualizar el saldo del usuario
    cursor.execute('UPDATE users SET balance = balance + ? WHERE username = ?', (amount, username))
    conn.commit()

    print("Deposit successful")

def withdraw():
    # Solicitar al usuario que ingrese la suma a retirar
    amount = float(input("Enter the amount to withdraw: "))

    # Comprobar si el retiro es valido
    cursor.execute('SELECT balance FROM users WHERE username = ?', (username,))
    balance = cursor.fetchone()[0]

    if amount > balance:
        print("Insufficient funds")
    else:
        
        # Actializar el saldo del usuario
        cursor.execute('UPDATE users SET balance = balance - ? WHERE username = ?', (amount, username))
        conn.commit()
        print("Withdrawal successful")

def balance_inquiry():
    
    # Recuperar el saldo del usuario
    cursor.execute('SELECT balance FROM users WHERE username = ?', (username,))
    balance = cursor.fetchone()[0]
    print("Your balance is:", balance)

def fund_transfer():
    
    # Solicitar al usuario el nombre de usuario del destinatario y el valor a transferir
    recipient = input("Enter the recipient's username: ")
    amount = float(input("Enter the amount to transfer: "))

    # Verificar si el valor de la transferencia es valido
    cursor.execute('SELECT balance FROM users WHERE username = ?', (username,))
    balance = cursor.fetchone()[0]

    if amount > balance:
        print("Insufficient funds")
    else:
        
        # Actualizar el saldo del usuario que realiza el envío
        cursor.execute('UPDATE users SET balance = balance - ? WHERE username = ?', (amount, username))
        
        # Actualizar el saldo del usuario que recibe el dinero
        cursor.execute('UPDATE users SET balance = balance + ? WHERE username = ?', (amount, recipient))
        conn.commit()
        print("Transfer successful")

# Main principal
while True:
    if not login():
        continue

    while True:
        
        # Mostrar las opciones del menu
        print("\nMenu:")
        print("1. Deposit")
        print("2. Withdraw")
        print("3. Balance Inquiry")
        print("4. Fund Transfer")
        print("5. Logout")

        # Preguntar al usuario la opción a elegir
        choice = input("Enter your choice: ")

        if choice == "1":
            deposit()
        elif choice == "2":
            withdraw()
        elif choice == "3":
            balance_inquiry()
        elif choice == "4":
            fund_transfer()
        elif choice == "5":
            print("Logged out")
            break
        else:
            print("Invalid choice")

KeyboardInterrupt: Interrupted by user